# RAG 接入Agent

## 配置llm和embedding模型

In [1]:
# -------------------------------------------
#     仍和第一节一样配置llm和embedding
# ------------------------------------------- 


import os
from dotenv import load_dotenv
ROOT_DIR= os.path.dirname(os.getcwd())
print(ROOT_DIR)
# 加载环境变量
load_dotenv()
print(os.getenv('ZISHU_API_KEY'))
# 从环境变量中读取api_key
api_key = os.getenv('ZISHU_API_KEY')
chat_model = "glm-4-flash"
emb_model = "embedding-2"



# -----------------------------------------------------
# 前面和之前一样，不过下面的llm创建不一样了


# 配置对话模型
from llama_index.llms.zhipuai import ZhipuAI
llm = ZhipuAI(
    api_key = api_key,
    model = chat_model,
)
#

# 测试对话模型
response = llm.complete("你是谁？")
print(response)

# 配置嵌入模型
from llama_index.embeddings.zhipuai import ZhipuAIEmbedding
embedding = ZhipuAIEmbedding(
    api_key = api_key,
    model = emb_model,
)


# 测试嵌入模型
emb = embedding.get_text_embedding("你好呀呀")
print(emb)
len(emb), type(emb)

c:\Users\zata\Desktop\wow\01-wow-agent
9d9338dcf6dcc6d703614ba53fce2e26.AdQUvVowRBvlh1cR
我是一个人工智能助手，名为 ChatGLM，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。
[-0.03837227, -0.00019120687, -0.010714491, -0.015073115, 0.016608356, -0.050502937, -0.025533188, -0.027715547, 0.021502344, 0.032557108, -0.03067428, -0.02086279, 0.0046875053, 0.034516953, -0.042270742, 0.041547254, -0.019662227, 0.026853487, -0.005376031, 0.009594618, -0.018981915, -0.013965981, -0.018312378, 0.0031198212, 0.04439648, -0.015677191, 0.009684721, -0.06881334, -0.047387555, -0.0016778187, 0.03813079, 0.0013801784, -0.04599552, 0.019898636, 0.018735, 0.027104413, 0.018217556, 0.042128537, 0.04531775, 0.03019625, -0.023391131, -0.01604703, -0.015947886, -0.0049376804, 0.044406954, -0.027735537, 0.055833668, 0.009265803, -0.018208269, -0.026014157, -0.016125116, -0.01624468, 0.02994953, 0.0187377, 0.020882482, 0.0121635, -0.040657587, -0.0013120841, -0.01602421, 0.048642263, -0.02346143, 0.0

(1024, list)

## 构建索引

In [2]:
# 从指定文件读取，输入为List
from llama_index.core import SimpleDirectoryReader,Document
documents = SimpleDirectoryReader(input_files=['./docs/问答手册.txt']).load_data()

# 构建节点
from llama_index.core.node_parser import SentenceSplitter
transformations = [SentenceSplitter(chunk_size = 512)]

from llama_index.core.ingestion.pipeline import run_transformations
nodes = run_transformations(documents, transformations=transformations)

# 构建索引
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss
from llama_index.core import StorageContext, VectorStoreIndex

emb = embedding.get_text_embedding("你好呀呀")
vector_store = FaissVectorStore(faiss_index=faiss.IndexFlatL2(len(emb)))
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes = nodes,
    storage_context=storage_context,
    embed_model = embedding,
)

In [3]:
# 构建检索器
from llama_index.core.retrievers import VectorIndexRetriever
# 想要自定义参数，可以构造参数字典
kwargs = {'similarity_top_k': 5, 'index': index, 'dimensions': len(emb)} # 必要参数
retriever = VectorIndexRetriever(**kwargs)

# 构建合成器
from llama_index.core.response_synthesizers  import get_response_synthesizer
response_synthesizer = get_response_synthesizer(llm=llm, streaming=True)

# 构建问答引擎
from llama_index.core.query_engine import RetrieverQueryEngine
engine = RetrieverQueryEngine(
      retriever=retriever,
      response_synthesizer=response_synthesizer,
        )

In [4]:
# 提问
question = "What are the applications of Agent AI systems ?"
response = engine.query(question)
for text in response.response_gen:
    print(text, end="")

Agent AI systems can be applied in various domains, including interactive AI for content generation, productivity tools for re-playing, paraphrasing, action prediction, and synthesizing scenarios, as well as in areas such as health care, gaming, and manufacturing. They can assist in the ethical deployment of large language models (LLMs) and versatile language models (VLMs), and contribute to the development of adaptive robotic systems. Additionally, they can focus on learning collaboration policies in simulations, which can have implications in the real world, such as in gaming and manufacturing industries.

In [5]:
# 配置查询工具
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import ToolMetadata
query_engine_tools = [
    QueryEngineTool(
        query_engine=engine,
        metadata=ToolMetadata(
            name="RAG工具",
            description=(
                "用于在原文中检索相关信息"
            ),
        ),
    ),
]

In [6]:
# 创建ReAct Agent
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [13]:
# 让Agent完成任务
# response = agent.chat("请问商标注册需要提供哪些文件？")
response = agent.chat("What are the applications of Agent AI systems ?")
print(response)

> Running step a4bdeb98-e989-425d-b86a-f49c489e2f08. Step input: What are the applications of Agent AI systems ?
Thought: The user is asking about the applications of Agent AI systems. I can use the RAG tool to find information on this topic.
Action: RAG工具
Action Input: {'input': 'Agent AI systems applications'}
Observation: Agent AI systems applications encompass diverse areas such as interactive AI, content generation for bots, productivity enhancements, and simulations of 3D and 2D scenarios. They can assist in re-playing, paraphrasing, action prediction, and synthesis of various types of content. Additionally, they have potential in health care, gaming, and manufacturing industries, where they can refine learning processes and redefine roles, necessitating new skill sets and responsible navigation of transitions to minimize socio-economic disruptions.
> Running step 07a824f8-fe8e-4135-8f68-f2e62fce9f83. Step input: None
Thought: I have gathered some information on the applications 

# 搜索引擎Agent

In [1]:
# -------------------------------------------
#     仍和第一节一样配置llm和embedding
# ------------------------------------------- 


import os
from dotenv import load_dotenv
ROOT_DIR= os.path.dirname(os.getcwd())
print(ROOT_DIR)
# 加载环境变量
load_dotenv()
print(os.getenv('ZISHU_API_KEY'))
# 从环境变量中读取api_key
api_key = os.getenv('ZISHU_API_KEY')
chat_model = "glm-4-flash"
emb_model = "embedding-2"



# -----------------------------------------------------
# 前面和之前一样，不过下面的llm创建不一样了


# 配置对话模型
from llama_index.llms.zhipuai import ZhipuAI
llm = ZhipuAI(
    api_key = api_key,
    model = chat_model,
)
#

# 测试对话模型
response = llm.complete("你是谁？")
print(response)

# 配置嵌入模型
from llama_index.embeddings.zhipuai import ZhipuAIEmbedding
embedding = ZhipuAIEmbedding(
    api_key = api_key,
    model = emb_model,
)


# 测试嵌入模型
emb = embedding.get_text_embedding("你好呀呀")
print(emb)
len(emb), type(emb)

c:\Users\zata\Desktop\wow\01-wow-agent
9d9338dcf6dcc6d703614ba53fce2e26.AdQUvVowRBvlh1cR
我是一个名为 ChatGLM 的人工智能助手，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。
[-0.03837227, -0.00019120687, -0.010714491, -0.015073115, 0.016608356, -0.050502937, -0.025533188, -0.027715547, 0.021502344, 0.032557108, -0.03067428, -0.02086279, 0.0046875053, 0.034516953, -0.042270742, 0.041547254, -0.019662227, 0.026853487, -0.005376031, 0.009594618, -0.018981915, -0.013965981, -0.018312378, 0.0031198212, 0.04439648, -0.015677191, 0.009684721, -0.06881334, -0.047387555, -0.0016778187, 0.03813079, 0.0013801784, -0.04599552, 0.019898636, 0.018735, 0.027104413, 0.018217556, 0.042128537, 0.04531775, 0.03019625, -0.023391131, -0.01604703, -0.015947886, -0.0049376804, 0.044406954, -0.027735537, 0.055833668, 0.009265803, -0.018208269, -0.026014157, -0.016125116, -0.01624468, 0.02994953, 0.0187377, 0.020882482, 0.0121635, -0.040657587, -0.0013120841, -0.01602421, 0.048642263, -0.02346143, 0.

(1024, list)

In [2]:
from llama_index.core.tools import FunctionTool
import requests
# 需要先把BOCHA_API_KEY填写到.env文件中去。
BOCHA_API_KEY = os.getenv('BOCHA_API_KEY')
print(BOCHA_API_KEY)
# 定义Bocha Web Search工具
def bocha_web_search_tool(query: str, count: int = 8) -> str:
    """
    使用Bocha Web Search API进行联网搜索，返回搜索结果的字符串。
    
    参数:
    - query: 搜索关键词
    - count: 返回的搜索结果数量

    返回:
    - 搜索结果的字符串形式
    """
    url = 'https://api.bochaai.com/v1/web-search'
    headers = {
        'Authorization': f'Bearer {BOCHA_API_KEY}',  # 请替换为你的API密钥
        'Content-Type': 'application/json'
    }
    data = {
        "query": query,
        "freshness": "noLimit", # 搜索的时间范围，例如 "oneDay", "oneWeek", "oneMonth", "oneYear", "noLimit"
        "summary": True, # 是否返回长文本摘要总结
        "count": count
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        # 返回给大模型的格式化的搜索结果文本
        # 可以自己对博查的搜索结果进行自定义处理
        return str(response.json())
    else:
        raise Exception(f"API请求失败，状态码: {response.status_code}, 错误信息: {response.text}")

search_tool = FunctionTool.from_defaults(fn=bocha_web_search_tool)
from llama_index.core.agent import ReActAgent
agent = ReActAgent.from_tools([search_tool], llm=llm, verbose=True)

sk-c004babfc7b14d17a2d3f3c445ac135f


In [3]:
# 测试用例
query = "阿里巴巴2024年的ESG报告主要讲了哪些内容？"
response = agent.chat(f"请帮我搜索以下内容：{query}")
print(response)

> Running step 850e7d06-0fae-471b-94ce-2ead4db83831. Step input: 请帮我搜索以下内容：阿里巴巴2024年的ESG报告主要讲了哪些内容？
Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: bocha_web_search_tool
Action Input: {'query': '阿里巴巴2024年ESG报告内容', 'count': 8}
Observation: {'code': 200, 'log_id': '708dc7ff5a8e30d0', 'msg': None, 'data': {'_type': 'SearchResponse', 'queryContext': {'originalQuery': '阿里巴巴2024年ESG报告内容'}, 'webPages': {'webSearchUrl': 'https://bochaai.com/search?q=阿里巴巴2024年ESG报告内容', 'totalEstimatedMatches': 80, 'value': [{'id': 'https://api.bochaai.com/v1/#WebPages.0', 'name': '阿里巴巴发布2024年ESG报告\u3000持续推进减碳与数字化普惠', 'url': 'https://www.alibabagroup.com/document-1752073403914780672', 'displayUrl': 'https://www.alibabagroup.com/document-1752073403914780672', 'snippet': '阿里巴巴集团发布《2024财年环境、社会和治理（ESG）报告》（下称“报告”），详细分享过去一年在ESG各方面取得的进展。报告显示，阿里巴巴扎实推进减', 'summary': '阿里巴巴集团发布《 2024 财年环境、社会和治理（ ESG ）报告 》（下称“报告”），详细分享过去一年在 ESG 各方面取得的进展。报告显示，阿里巴巴扎实推进减碳举措，全集

智谱AI的GLM-4模型进行联网搜索，返回搜索结果的字符串。

In [6]:
from zhipuai import ZhipuAI
from datetime import datetime
# 定义Zhipu Web Search工具
def zhipu_web_search_tool(query: str) -> str:
    """
    使用智谱AI的GLM-4模型进行联网搜索，返回搜索结果的字符串。
    
    参数:
    - query: 搜索关键词

    返回:
    - 搜索结果的字符串形式
    """
    # 初始化客户端
    client = ZhipuAI(api_key=api_key)

    # 获取当前日期
    current_date = datetime.now().strftime("%Y-%m-%d")

    print("current_date:", current_date)
    
    # 设置工具
    tools = [{
        "type": "web_search",
        "web_search": {
            "enable": True
        }
    }]

    # 系统提示模板，包含时间信息
    system_prompt = f"""你是一个具备网络访问能力的智能助手，在适当情况下，优先使用网络信息（参考信息）来回答，
    以确保用户得到最新、准确的帮助。当前日期是 {current_date}。"""
        
    # 构建消息
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query}
    ]
        
    # 调用API
    response = client.chat.completions.create(
        model="glm-4-flash",
        messages=messages,
        tools=tools
    )
    
    # 返回结果
    return response.choices[0].message.content

rst = zhipu_web_search_tool("2025年为什么会闰六月？")
print(rst)

current_date: 2025-02-14
2025年实际上并没有闰六月。闰月是中国农历中的概念，用于调整农历与公历之间的差异。在中国农历中，每19年会有7个闰月，这些闰月通常插入到19年周期的某些月份之间，以保持农历与天文年的同步。

然而，闰月的插入并不是每年都固定发生的。2025年并没有插入闰月。根据中国天文学会发布的农历数据，2025年的农历中没有闰月，所以2025年只有12个月份。

如果有人声称2025年有闰六月，那可能是基于错误的假设或误解。闰月的确定是基于复杂的历法计算，需要专业的天文学知识。
